## About this code

This script apply entire pipeline to new batch of tweets:<br>
1. Pre-process
2. Feature engineering step-1 (low computation)
3. Relevance classification (0/1-irrelevant)
4. Filter non relevant tweets
5. Feature engineering step-2 (high computation)
6. Support classification (-1/0/1)
7. Subject classification (0/1-political)
8. Virality prediction (regression)

Imports

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import csv
from tqdm import tqdm

In [2]:
#### Import thesis modules
os.chdir('../relevance algo')
import relevance_prediction
os.chdir('../support algo')
import support_prediction
os.chdir('../user_support algo')
import user_support_prediction

/anaconda3/envs/thsis/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
# Avoid trimming text in preview
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

np.random.seed(20170805)
tqdm.pandas()

## Run configurations

<div class="alert alert-block alert-success">
'min_couple_appearances': 0<br>
'feature_selection': 'False', # When predicting new data - no auto features selection is applied.<br> 
                              #Specific features are selected by model needs.
</div>

In [4]:
# ## Run configurations

# 'min_couple_appearances': 0<br>
# 'feature_selection': 'False', # When predicting new data - no auto features selection is applied.<br> 
#                               #Specific features are selected by model needs.
def get_config_object():
    config = {
                # What to predict
                'predict_relevance': True,
                'predict_support': True,
                'predict_user_support': True,
                'predict_subject': False,
                'support_model': 'RF', # RF or XGboost
                'user_support_model': 'RF',  # RF or XGboost

                # If to split the data to chunks because of data size
                'split_data_to_chunks': False,
                'chunk_size': 10000, # Number of rows in df per chunk
                # Distributed pipeline
                'distributed_pipeline': False, # If the pipeline proccess (relevance & support (additional features)) should be distributed using workers pooling
                'n_workers': 8,

                # Mode
                'learning': False,           # True at the train phase, False when operational - predict new data using existing model

                # Data
                'filter_taggers': False,         # If to filter labeled data by the tagger - Yogev, Ofir and Itay
                'target': 'relevance',           # The target of the algorithm
                'load_df_from_pickle': False,    # If to load a complete data-frame or to execute the entire pipeline from the beginning
                'twitter_foundation_date': '2006-03-21 12:00:00',
                'data_extraction_time': '2018-03-01 12:00:00',

                # Pre-process
                'slang': True,              # If to handle slang words, also for Feature engineering
                'spell_correction': False,  # If to use spell correction in the pre-proccess phase
                'col': 'text',              # Column to apply pre-proccess on - used by the proccess_tweet function in tweet_pre_proccess.py

                # Feature engineering
                'nlp_features': True,
                'word_type': False,          # Classes - verbs, nouns, etc.
                'dominant_keywords': True,
                'user_features': False,
                'time_and_event': False,
                'nlp_raw': True,
                'emotion': False,
                'hashtags_and_mentions': True,
                # 'num_dominant_words': 15,   # Number of dominant words to use in the NLP features (dominant_keywords function - parameter k)
                'min_couple_appearances': 0,# For couple of dominant words -
                                            # how many time the combination need to appear in the data in order to become a feature
                'url_features': False,
                'Tweets_media_and_content': True,
                'country_support': False,
                'entities_features': True,
                'word_type': False,         #word class dict feature. will be false for the relevance classification
                'sentiment': False,          # NLTK sentiment feature

                # Feature selection
                'feature_selection': False, # When predicting new data - no auto features selection is applied.
                                              # Specific features are selected by model needs.
                'remove_features_zero_variance': False,
                'remove_correlated_features': False,
                'remove_low_correlated_features': False,
                'feature_importance': False,
                # 'corr_per_thresh': 0.05,    # Percentile threshold of Min correlation between a feature and the target variable (abs)
                # 'importance_per_thresh': 10,# Percentile threshold of Min importance of a feature
                # 'PCA': False,               # If the use PCA to reduce dimensiality
                # 'PCA_var': 0.995,           # % of commulative explained variance required from PCA (affect #of PC)

                # Model
                'load_model_pickle': True,  # If to import existing models from pickle
                'regression': False,        # If to run regression model also
                'validation': False,        # If to use train split to train and validation
                'class_threshold': 0,       # Probability threshold in order to classify a tweet as negative/neurtal/positive
                'bench_from_pickle': False
        }

    config['only_tweets_with_emotions'] = True # Classify only Tweets with IBM emotions -> affect the merge command between emotion_df and full_df before support algo

    return config

In [5]:
def run_pipeline(data, prediction_file_suffix=""):
    # ## 1. Relevance Algo
    #     - df_relevance_no_filter_no_prediction = no filtered features, no prediction<br>
    #     - df_relevance_no_filter_w_prediction.tsv = all features + relevance prediction<br>
    #     - data_w_relevance = selected features for relevance algo + no prediction
    config = get_config_object()

    #### Run prediction
    print('########################### RELEVANCE Prediction ###########################' )
    df, full_df = relevance_prediction.predict(data, config)

    ##### Adding relevance feature to original data
    data = relevance_prediction.add_relevance(df,data)

    #### Remove irrelevant tweets
    # Filter irrelevant tweets using the relevance label/prediction
    full_df = relevance_prediction.remove_irrelevants(df, full_df) if not config['load_df_from_pickle'] else full_df

    # ## 2. Support Algo
    # Add emotion analysis by watson IBM to tweets (if available)
    emotion_df = pd.read_pickle('../../data/classifiers/emotion_df.pkl')
    columns = list(full_df.columns)+['emotion']
    full_df = full_df.merge(emotion_df, on='text', how='inner' if config['only_tweets_with_emotions'] else 'left', suffixes=('','_emotion'))

    # Select desired features only
    full_df = full_df[columns]

    if config.get('predict_support', True) or config.get('predict_subject', True) or config.get('predict_user_support', True):
        if config.get('predict_support', True):
            print('########################### SUPPORT Prediction ###########################' )
            # #### Change configuration
            # Change configuration input
            config['target'] = 'support'
            config['sentiment'] = True
            config['emotion'] = True
            config['country_support'] = True
            config['word_type'] = True
            config['load_df_from_pickle'] = False
            # config['user_features'] = True
            config['user_bio'] = True
            config['time_and_event'] = True  # Time and event features
            config['twitter_foundation_date'] = '2006-03-21 12:00:00'
            config['network_features'] = False
            config['load_network_data'] = True

            # #### Adding support features
            if config['distributed_pipeline'] and not config['load_df_from_pickle']:
                print(' - Distributed to {} workers'.format(config['n_workers']))
                full_df = support_prediction.mp_handler(full_df, config)
            else:
                full_df = support_prediction.adding_features(full_df, config) # if not config['load_df_from_pickle'] else pd.read_pickle('df_pipeline_support_all_features.tsv', header=0, sep='\t')
                print('Support features loaded')

            if config.get('predict_support', True):
                # #### Run prediction
                # Output to 'support_df' - the data frame for the support algorithm (filtered features + support prediction)
                print('\nPREDICTING: Tweet political leaning')
                support_df, full_df = support_prediction.predict(full_df, config)
                print(' - COMPLETED')

                # ##### Adding support feature to original data
                print('Adding support prediction feature')
                data = support_prediction.add_support(support_df, data)
                full_df = support_prediction.add_support(support_df, full_df)
                print(' - COMPLETED')

        if config.get('predict_user_support',True):
            print('########################### USER SUPPORT Prediction ###########################')
            # #### Change configuration
            # Change configuration input
            # # If features were added before (their config is TRUE) -> turn to False
            config['target']                = 'user_support'
            config['sentiment']             = True if not config['sentiment'] else False
            config['emotion']               = True if not config['emotion'] else False
            config['country_support']       = True if not config['country_support'] else False
            config['word_type']             = True if not config['word_type'] else False
            config['network_features']      = True if not config['network_features'] else False
            config['user_bio']              = True if not config['user_bio'] else False
            config['load_df_from_pickle'] = False
            config['time_and_event'] = False  # Time and event features
            config['twitter_foundation_date'] = '2006-03-21 12:00:00'
            # config['load_network_data'] = True

            # #### Adding support features
            if config['distributed_pipeline'] and not config['load_df_from_pickle']:
                print(' - Distributed to {} workers'.format(config['n_workers']))
                full_df = user_support_prediction.mp_handler(full_df, config)
            else:
                full_df = user_support_prediction.adding_features(full_df,config)
                    # if not config['load_df_from_pickle'] else pd.read_pickle('df_pipeline_support_all_features.tsv', header=0, sep='\t')
                print('User Support features loaded')

            # #### Run prediction
            # Output to 'user_support_df' - the data frame for the support algorithm (filtered features + support prediction)
            print('\nPREDICTING: User political leaning by Tweets')
            support_df, full_df = user_support_prediction.predict(full_df, config)
            print(' - COMPLETED')

            # ##### Adding support feature to original data
            print('Adding support prediction feature')
            data = user_support_prediction.add_support(support_df, data)
            full_df = user_support_prediction.add_support(support_df, full_df)
            print(' - COMPLETED')

    if config.get('predict_subject', True):
        print('########################### Subject Prediction ###########################')
        # ## 3. Subject Algo
        os.chdir('../subject algo')
        # Change config input
        config['target'] = 'relevance'
        config['subject'] = True
        import subject_prediction
        subject_df = full_df.copy()

        # #### Update dominant words features
        subject_df = subject_prediction.adding_features(subject_df, config)

        # #### Run prediction
        print('PREDICTING: Tweet subject')
        subject_df = subject_prediction.predict(subject_df, config)
        print(' - COMPLETED')

        ##### Adding subject feature to original data
        print('Adding support prediction feature')
        full_df = subject_prediction.add_subject(subject_df,full_df)
        print(' - COMPLETED')

    # ### Export results
    print('\nExporting final results...')
    if config['split_data_to_chunks']:
        full_df.to_csv('../../data/predictions/df_full_pipeline_predictions_{}.tsv'.format(str(datetime.today()).split(' ')[0], prediction_file_suffix), header=True, sep='\t')
        full_df[list(data.columns)].to_csv('../../data/predictions/data_pipeline_predictions_{}_{}.tsv'.format(str(datetime.today()).split(' ')[0], prediction_file_suffix), header=True, sep='\t')

    else:
        full_df.to_csv('../../data/predictions/df_full_pipeline_predictions_{}.tsv'.format(str(datetime.today()).split(' ')[0]), header=True,sep='\t')
        full_df[list(data.columns)].to_csv('../../data/predictions/data_pipeline_predictions_{}.tsv'.format(str(datetime.today()).split(' ')[0]), header=True,sep='\t')

    print('\nONLINE PIPELINE COMPLETED')


In [6]:
data = pd.read_csv("../../data/domain_full_data.tsv", header=0, encoding='utf-8', sep='\t',
                   quoting=csv.QUOTE_NONE).iloc[:, 1:]

In [7]:
config = get_config_object()

In [8]:
if config['only_tweets_with_emotions']:
    print('Num tweets', data.shape[0])
    emotion_df = pd.read_pickle('../../data/classifiers/emotion_df.pkl')
    tweets_with_emotion = set(emotion_df[emotion_df.emotion.notna()].text.tolist())
    data = data[data.text.isin(tweets_with_emotion)]
    print('Only Tweets with emotions (filter is activated)', data.shape[0])
    print()

Num tweets 781734
Only Tweets with emotions (filter is activated) 174740



In [9]:
########################################################################################################################
###################################################### RUN PIPELINE ####################################################
########################################################################################################################

if config['split_data_to_chunks']:
    # Run pipeline on the data object in chunks
    chunks = data.groupby(np.arange(len(data)) // config['chunk_size'])
    for chunk_num, df in tqdm(chunks):
        print('')
        run_pipeline(df, "chunk_{}".format(chunk_num))
else:
    run_pipeline(data)

########################### RELEVANCE Prediction ###########################
Creating features for relevance algo
----------------------
  - Lower case completed
  - Strip punctuation completed
  - Stop-words removal completed
  - Tokenization completed
  - Slang words correction completed
  - Stemming phase completed
  - Cleaning empty tokens completed
Final number of features: 174740

COMPLETED: Pre-proccess
----------------------
----------------------
START: NLP features
 Raw NLP features
  - Word classes features doesnt need to be done (1/5)
  - Capital letters features completed (2/5)


  0%|          | 0/174740 [00:00<?, ?it/s]

  - 1 length words features completed (3/5)
  - Simple features completed (4/5)


100%|██████████| 174740/174740 [00:00<00:00, 488025.10it/s]


  - Negation words feature completed (5/5)

 Hashtags & Mentions
  - Mentions feature completed
  - Hashtags features completed

 Dominant keywords
 - loading dominant words
 - Creating Bag-of-Words features
 - num_dominant_words
 - Sentiment * dominant words
COMPLETED: Dominant words

 START: Entities features
 - COMPLETED

 Tweet's media and content features completed

COMPLETED: NLP features
----------------------
----------------------
Feature engineering COMPLETED
Pipeline COMPLETED
Pipeline results loaded from tsv
 - COMPLETED
Predicting relevance
 - COMPLETED
COMPLETED: Relevance prediction
Adding relevance feature
Removing irrelevants tweets
 - COMPLETED
# of relevant tweets: 165179. 9561 tweets have been removed
########################### SUPPORT Prediction ###########################
Removing dominant words features of previous algorithm
 - COMPLETED
Building features for support algo
  - START: Country support


...

Unclassified tweets: 0


/Users/ofirmagdaci/thesis-code/thesis-classifiers/user_support algo/user_support_prediction.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['user_support_weighted'] = predictions['prediction_weighted']


 - COMPLETED
Adding support prediction feature


/Users/ofirmagdaci/thesis-code/thesis-classifiers/user_support algo/user_support_prediction.py:143: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([relevant_data, irrelevant_data], axis=0).reset_index(drop=True)


 - COMPLETED

Exporting final results...

ONLINE PIPELINE COMPLETED
